In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from IPython import embed
import pickle
from torch.utils.data import DataLoader, Dataset

In [2]:
data = pickle.load(open("games.pkl", "rb"))
len(data)
train_df = data[:10000]
test_df = data[10000:12000]
print(len(train_df))
print(len(test_df))

10000
2000


In [3]:
print(test_df.iloc[0:10]) 

       turns winner                                   moves
10000     19  white   e4 e5 d3 d6 Nf3 Nf6 Nh4 Nh5 Qxh5 Qxh4
10001     16  black     Nc3 e5 Nf3 Nf6 b3 d5 g3 Ne4 Na4 Bg4
10002     45  white    d4 e6 e4 d6 Nf3 Kd7 Ng5 Qxg5 Bxg5 f6
10003     10  black  b4 e5 Na3 Nf6 Bb2 Ne4 Rb1 Qh4 e3 Qxf2#
10004     21  white     e4 g6 d4 Nh6 e5 Bg7 Bxh6 Bxh6 g3 a5
10005     21  white  e4 g6 Nf3 b6 Ng5 Nf6 Bc4 Nxe4 Nxf7 Ng5
10006     87  white        e4 h5 d4 g6 f4 Bh6 c4 Nf6 e5 Ng8
10007     65  white   d4 Nc6 e3 Nb4 c3 Nd5 Bc4 Ndf6 Qf3 Nh6
10008     39  black     g4 d5 b3 Bxg4 h3 Bf5 Nc3 Nf6 Nb5 c5
10009     38  black       g3 c6 Bh3 g6 e3 Bg7 Bg2 a5 a3 Na6


In [4]:
keyword = 'O-O'
train_df['moves']

train_df.loc[323, 'moves'].split()
any(keyword in x for x in train_df.loc[323, 'moves'].split())

True

In [5]:
zeroth_move = train_df.loc[1, 'moves'].split()[0]  # zeroth move in game 1
print(zeroth_move)
print(type(zeroth_move))

d4
<class 'str'>


In [6]:
test_game = train_df.loc[28, 'moves'].split()

In [7]:
# this method takes in all the moves in a game and encodes player, piece, file, rank, and modifiers
def move_vec(moves):

    move_vectors = np.ones((10,5), dtype=int) * -1
    
    player_idx = 0
    piece_idx = 1
    file_idx = 2
    rank_idx = 3
    mod_idx = 4

    piece_map = {'pawn': 0, 'N': 1, 'B': 2, 'R': 4, 'Q': 5, 'K': 6}
    file_map = {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8}
    rank_list = np.arange(1,9)
    mod_map = {'O-O-O': 0, 'O-O': 1, 'x': 2, '+': 3, '#': 4, '=': 5} # queenside castle, kingside castle, capture, check, mate, promo
    
    for i, move in enumerate(moves):
        if(i%2==0):                            # encode PLAYER as index 0
            move_vectors[i][player_idx] = 0    # white move
        else:
            move_vectors[i][player_idx] = 1    # black move
            
            
        for key in piece_map:                  # encode PIECE as index 1 
            if key in move:
                move_vectors[i][piece_idx]=piece_map[key]
                
        if move_vectors[i][piece_idx] == 100:
            move_vectors[i][piece_idx] = 0    # set pawns
                
                
        for key in file_map:                   # encode FILES A-B as integers 1-8
            if key in move:
                move_vectors[i][file_idx] = file_map[key]
                
                
        for rank in rank_list:                 # encode RANK 1-8 as integers 1-8
            if str(rank) in move:
                move_vectors[i][rank_idx] = int(rank)   

                
        for key in mod_map:                    # encode queenside castle, kingside castle, capture, check, mate, promo
            if key in move:
                move_vectors[i][mod_idx] = mod_map[key]

    return move_vectors

test = move_vec(test_game)

In [9]:
print("color,\npiece: 'pawn': 0, 'N': 1, 'B': 2, 'R': 4, 'Q': 5, 'K': 6")
print("file:'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8")
print("moves string: ")
print(test_game)
test

color,
piece: 'pawn': 0, 'N': 1, 'B': 2, 'R': 4, 'Q': 5, 'K': 6
file:'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8
moves string: 
['e4', 'c5', 'Nf3', 'd6', 'Bb5+', 'Bd7', 'Bxd7+', 'Nxd7', 'O-O', 'Ngf6']


array([[ 0, -1,  5,  4, -1],
       [ 1, -1,  3,  5, -1],
       [ 0,  1,  6,  3, -1],
       [ 1, -1,  4,  6, -1],
       [ 0,  2,  2,  5,  3],
       [ 1,  2,  4,  7, -1],
       [ 0,  2,  4,  7,  3],
       [ 1,  1,  4,  7,  2],
       [ 0, -1, -1, -1,  1],
       [ 1,  1,  7,  6, -1]])

In [ ]:
# make sure data types are consistent

dt_arr = np.empty_like(test, dtype=object)
for i, val in np.ndenumerate(test):
    dt_arr[i] = str(type(val))

print(dt_arr[0])

In [ ]:
# i need samples and labels. samples are the moves, labels are the winners.
# but first, am i balanced? 

num_black_wins = (train_df['winner'] == 'black').sum()
num_black_wins_test = (test_df['winner'] == 'black').sum()

print("Number of trainset where black wins:", num_black_wins, "out of", len(train_df), ": ", num_black_wins / len(train_df))
print("Number of testset where black wins:", num_black_wins_test, "out of", len(test_df), ": ", num_black_wins_test / len(test_df))

In [ ]:
from torch.utils.data import DataLoader, Dataset

# Define your custom dataset class
class ChessDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Convert data to PyTorch tensors
        X = torch.from_numpy(self.data.iloc[idx, :-1].values).float()
        y = torch.from_numpy(np.array(self.data.iloc[idx, -1])).long()
        
#         X = torch.from_numpy(self.data.iloc[idx, :-1]).float()
#         y = torch.from_numpy(np.array(self.data.iloc[idx, -1])).long()
        
        return X, y

# Create train and test data loaders
train_data = ChessDataset(train_df)
test_data = ChessDataset(test_df)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
class ChessRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ChessRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        out, hidden = self.rnn(x)
        out = self.fc(hidden[-1])
        
        return out

In [ ]:
# Set hyperparameters
input_size = 12 # number of features in the input (one-hot encoding of moves)
hidden_size = 128 # number of hidden units in the RNN
output_size = 2 # number of output classes (1 for each player)

learning_rate = 0.001
num_epochs = 10

model = ChessRNN(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
for epoch in range(num_epochs):
    train_loss = 0.0
    train_acc = 0.0
    
    # Set model to train mode
    model.train()
    
    # Loop over the train data
    for i, (inputs, labels) in enumerate(train_loader):
        # Zero out gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Update metrics
        train_loss += loss.item() * inputs.size(0)
        train_acc += torch.sum(torch.argmax(outputs, axis=1) == labels).item()
        
    # Compute average loss and accuracy for the epoch
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = train_acc / len(train_loader.dataset)
    
    # Print training metrics for the epoch
    print('Epoch [{}/{}], Train Loss: {:.4f}, Train Acc: {:.4f}'.format(
        epoch+1, num_epochs, train_loss, train_acc))
